In [10]:
import requests
import json
import praw
import time
import boto3
import pandas as pd

def avg_comment_sentiment(comment_list):
    '''
    MUST HANDLE TOP MODERATOR COMMENT BEFORE CALLING
    Params:
        List of comment objects
    Returns:
        Weighted average of comments
    '''
    client = boto3.client('comprehend')
    comment_sentiment_list = []
    for comment in comment_list:
        response = client.detect_sentiment(
            Text=comment.body[0:499],
            LanguageCode='en'
        )
        global calls
        calls += 5
        if response['Sentiment']=='NEUTRAL':
            sentiment = 0
        elif response['Sentiment']=='POSITIVE':
            sentiment = 1
        elif response['Sentiment']=='NEGATIVE':
            sentiment = -1
        else:
            sentiment = 0
        comment_sentiment_list.append(comment.ups * sentiment)
    return sum(comment_sentiment_list) / len(comment_sentiment_list)

def avg_comment_brands(comment_list):
    ''' 
    Params:
        List of comment objects
    Returns:
        Weighted average of brand count
    '''
    client = boto3.client('comprehend')
    comment_numbrands_list = []
    for comment in comment_list:
        response = client.detect_entities(
            Text=comment.body[0:499],
            LanguageCode='en'
        )
        global calls
        calls += 5
        organization_count = 0
        for ent in response['Entities']:
            if ent['Type'] == 'ORGANIZATION':
                #print(ent['Text'])
                organization_count += 1
        comment_numbrands_list.append(comment.ups * organization_count)
    return sum(comment_numbrands_list)/ len(comment_numbrands_list)

def title_sentiment(title):
    ''' 
    MUST HANDLE TOP MODERATOR COMMENT BEFORE CALLING
    Params:
        List of comment objects
    Returns:
        Weighted average of comments
    '''
    client = boto3.client('comprehend')
    comment_sentiment_list = []
    response = client.detect_sentiment(
        Text=title[0:499],
        LanguageCode='en'
    )
    global calls
    calls += 5
    if response['Sentiment']=='NEUTRAL':
        sentiment = 0
    elif response['Sentiment']=='POSITIVE':
        sentiment = 1
    elif response['Sentiment']=='NEGATIVE':
        sentiment = -1
    else:
        sentiment = 0
    return sentiment

def title_brands(title):
    ''' 
    Params:
        List of comment objects
    Returns:
        Weighted average of brand count
    '''
    client = boto3.client('comprehend')
    comment_numbrands_list = []
    response = client.detect_entities(
        Text=title[0:499],
        LanguageCode='en'
    )
    global calls
    calls += 5
    organization_count = 0
    for ent in response['Entities']:
        if ent['Type'] == 'ORGANIZATION':
            organization_count += 1
    return organization_count

def data_dict(submission):
    target_sub_ID=submission.id
    target_sub=reddit.submission(target_sub_ID)
    data_input={}
    data_input["Title Seniment"]=str(title_sentiment(target_sub.title))
    data_input["Title Brands"]=str(title_brands(target_sub.title))
    data_input["Author Link Karma"]=str(target_sub.author.link_karma)
    data_input["Author Comment Karma"]=str(target_sub.author.comment_karma)
    data_input["Author Age"]=str(((time.time()-target_sub.author.created_utc)/86400))
    commentList=[]
    target_sub.comments.replace_more(limit=0)
    i = 0
    for top_com in target_sub.comments:
        commentList.append(top_com)
        i += 1
        if i > 25:
            break
    data_input["Avg Comment Sentiment"]=str(avg_comment_sentiment(commentList))
    data_input["Avg Comment Brand Count"]=str(avg_comment_brands(commentList))
    global calls
    print(calls)
    return data_input

def api(post):
    URL = "https://2ut1lyn470.execute-api.us-east-1.amazonaws.com/beta"
    PARAMS = data_dict(post)
    r = requests.post(url = URL, data = json.dumps(PARAMS))
    data = r.json()
    return data

In [11]:
import praw
import time
import boto3
import pandas as pd

reddit=praw.Reddit(client_id='19fTPCCLQWpLTw',
                     client_secret='U8NzJi7xQaFJUruoG1yuJlgEr6w',
                     user_agent='hackuva ml project by u/cpgeier',
                     username='cpgeier',
                     password='Sploot24551')
print(reddit.user.me())

calls = 0
e_count=0
data_dict=[]
for submission in reddit.subreddit('HailCorporate').top(limit=1000):
    try:
        target_sub_ID=submission.id_from_url(submission.url)
        target_sub=reddit.submission(target_sub_ID)    
        if(target_sub.subreddit!="HailCorporate"):
            data_input={}
            data_input["PostID"]=target_sub.id
            print("On " + data_input["PostID"])
            data_input["Title Seniment"]=title_sentiment(target_sub.title)
            data_input["Title Brands"]=title_brands(target_sub.title)
            data_input["Author Link Karma"]=target_sub.author.link_karma
            data_input["Author Comment Karma"]=target_sub.author.comment_karma
            data_input["Author Age"]=(time.time()-target_sub.author.created_utc)/86400
            commentList=[]
            target_sub.comments.replace_more(limit=0)
            i = 0
            for top_com in target_sub.comments:
                commentList.append(top_com)
                i += 1
                if i > 20:
                    break
            data_input["Avg Comment Sentiment"]=avg_comment_sentiment(commentList)
            data_input["Avg Comment Brand Count"]=avg_comment_brands(commentList)
            data_dict.append(data_input)
            print(calls)
    except Exception as e:
        print("Excepted " + str(target_sub.id) + " " + str(e))
        print(calls)
        continue
        #data_input["Comments"]=commentList            

cpgeier
On 6mv4x5
220
On 7g2p09
Excepted 7g2p09 received 404 HTTP response
230
On 5xh2kh
450
On 6cpnq2
670
Excepted 6eyz8a received 403 HTTP response
670
On 4zx28b
890
On 3ulzdy
1110
Excepted 6lrr28 received 403 HTTP response
1110
On 6s5q40
1330
On 6nvtyr
1550
Excepted 6nvtyr Invalid URL: https://arstechnica.com/tech-policy/2017/11/comcast-deleted-net-neutrality-pledge-the-same-day-fcc-announced-repeal/
1550
Excepted 6nvtyr Invalid URL: http://www.nbcnews.com/business/unhappy-meal-mcdonalds-suit-claims-toddler-ate-used-condom-1C8876034
1550
On 6ccpbn
1770
On 6176ri
1990
On 6j0xi6
2210
On 60p3n1
2430
On 6orunj
Excepted 6orunj 'Redditor' object has no attribute 'link_karma'
2440
On 86ahf2
2660
On 7bf2zk
2880
On 6idoah
3100
On 6z6a34
3320
On 4zjc76
3540
On 6uplo4
3760
On 6402yl
3980
On 83ichd
4200
Excepted 83ichd Invalid URL: https://www.commondreams.org/views/2017/11/19/fake-news-only-beginning-fcc-votes-let-monopolies-decide-what-local-news-you-see
4200
On 7q2cl1
4420
On 7akhhq
4640
On 

37690
On 85aua7
37910
On 70wihs
38130
On 53ehep
38350
On 4gtafk
Excepted 4gtafk 'NoneType' object has no attribute 'link_karma'
38360
On 1b3jmx
38580
On 7qmgi7
38800
On 6nra06
Excepted 6nra06 'Redditor' object has no attribute 'link_karma'
38810
On 6hurma
39030
On 4youb1
39250
On 7by7xx
39470
On 6scr3z
Excepted 6scr3z 'Redditor' object has no attribute 'link_karma'
39480
On 4muip6
39700
On 6ku1n2
39920
On 6rlgnb
Excepted 6rlgnb 'NoneType' object has no attribute 'link_karma'
39930
Excepted 6rlgnb Invalid URL: http://www.adweek.com/news/technology/marketing-reddit-scary-these-success-stories-show-big-potential-168278
39930
On 6xg3hr
40150
On 5vq00v


KeyboardInterrupt: 

In [13]:
df = pd.DataFrame.from_dict(data_dict)

In [15]:
len(df)

183

In [17]:
df_x_true = pd.DataFrame.from_dict(data_dict)
df_x_true.head()
df_x_true.to_json("m_dataset.json")

In [19]:
post_subs = []
c = 0
print("WOW!")
for postid in df_x_true['PostID']:
    post_subs.append(str(reddit.submission(id=postid).subreddit))
    print(c)
    c += 1
print(post_subs)

WOW!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
['The_Donald', 'todayilearned', 'pics', 'undelete', 'explainlikeimfive', 'pics', 'todayilearned', 'funny', 'funny', 'todayilearned', 'announcements', 'Futurology', 'pics', 'pics', 'videos', 'funny', 'pics', 'PrequelMemes', 'mildlyinteresting', 'aww', 'gifsthatkeepongiving', 'todayilearned', 'funny', 'videos', 'xboxone', 'pics', 'legaladvice', 'todayilearned', 'xboxone', 

In [20]:
post_sub_value_counts = pd.Series(post_subs).value_counts()

In [22]:
calls = 0
e_count=0
data_dict=[]
cur_subreddit = 0
for post_sub_value in post_sub_value_counts:
    for submission in reddit.subreddit(post_sub_value_counts.index[cur_subreddit]).hot(limit=post_sub_value):
        print(post_sub_value_counts.index[cur_subreddit])
        try:
            #target_sub_ID=submission.id_from_url(submission.url)
            #target_sub=reddit.submission(target_sub_ID)
            target_sub = submission
            if(target_sub.subreddit!="HailCorporate"):
                data_input={}
                data_input["PostID"]=target_sub.id
                print("On " + data_input["PostID"])
                data_input["Title Seniment"]=title_sentiment(target_sub.title)
                data_input["Title Brands"]=title_brands(target_sub.title)
                data_input["Author Link Karma"]=target_sub.author.link_karma
                data_input["Author Comment Karma"]=target_sub.author.comment_karma
                data_input["Author Age"]=(time.time()-target_sub.author.created_utc)/86400
                commentList=[]
                submission.comments.replace_more(limit=0)
                i = 0
                for top_com in target_sub.comments:
                    commentList.append(top_com)
                    i += 1
                    if i > 50:
                        break
                data_input["Avg Comment Sentiment"]=avg_comment_sentiment(commentList)
                data_input["Avg Comment Brand Count"]=avg_comment_brands(commentList)
                data_dict.append(data_input)
                print('Succeeded: ' + data_input["PostID"])
                print(calls)
        except Exception as e:
            print("Excepted " + str(target_sub.id) + " " + str(e))
            print(calls)
            continue
        #data_input["Comments"]=commentList
    cur_subreddit += 1
df_x_false = pd.DataFrame.from_dict(data_dict)

pics
On 86tvyu
Succeeded: 86tvyu
420
pics
On 86y86c
Succeeded: 86y86c
940
pics
On 86y1g3
Succeeded: 86y1g3
1460
pics
On 86wvkx
Succeeded: 86wvkx
1980
pics
On 86wng5
Succeeded: 86wng5
2500
pics
On 86w82e
Succeeded: 86w82e
3020
pics
On 86xgdn
Succeeded: 86xgdn
3540
pics
On 86usg2
Succeeded: 86usg2
4060
pics
On 86v8f2
Succeeded: 86v8f2
4580
pics
On 86uoiy
Succeeded: 86uoiy
5100
pics
On 86xmdo
Succeeded: 86xmdo
5560
pics
On 86vd21
Succeeded: 86vd21
6080
pics
On 86x2u0
Succeeded: 86x2u0
6600
pics
On 86x6ri
Succeeded: 86x6ri
6840
pics
On 86v2sh
Succeeded: 86v2sh
7360
pics
On 86u1tv
Succeeded: 86u1tv
7880
pics
On 86ybf2
Succeeded: 86ybf2
8060
pics
On 86u1m0
Succeeded: 86u1m0
8580
pics
On 86szpj
Succeeded: 86szpj
9100
pics
On 86sohd
Succeeded: 86sohd
9620
pics
On 86xtay
Succeeded: 86xtay
9890
pics
On 86xhfz
Succeeded: 86xhfz
9950
pics
On 86tybx
Succeeded: 86tybx
10170
pics
On 86wq0n
Succeeded: 86wq0n
10360
pics
On 86vf1d
Succeeded: 86vf1d
10650
pics
On 86ub1c
Succeeded: 86ub1c
10940
pics
On 86

KeyboardInterrupt: 

In [23]:
df_x_false = pd.DataFrame.from_dict(data_dict)

In [24]:
df_x_false.to_json("df_x_false.json")

In [25]:
import pickle
df_x_true.to_pickle("df_x_true2.pk")
df_x_false.to_pickle("df_x_false2.pk")
df_x_true['Label'] = 1
df_x_false['Label'] = 0
df_x_true.head()
df = pd.concat([df_x_true,df_x_false])
df.head()

df_clean = df
df_clean = df_clean.drop(['PostID'],axis=1)
df_clean = df_clean.sample(frac=1)
df_clean.head()
df_clean.to_csv("dataset.csv",index=False)

In [28]:
df_clean.head()

,Author Age,Author Comment Karma,Author Link Karma,Avg Comment Brand Count,Avg Comment Sentiment,Title Brands,Title Seniment,Label
16,1788.078197,46501,66881,0.058824,0.000000,0,0,0
39,970.786250,72097,106371,24.686275,-9.019608,0,0,0
103,2294.243970,253,5294,26.095238,18.904762,0,0,1
177,1195.456801,113492,19801,1086.619048,120.190476,2,0,1
14,1056.744490,62113,26595,25.523810,-755.714286,1,0,1


In [29]:
len(df_clean)

291

In [109]:
a = api(s)

14046


In [110]:
print(a)

{'Prediction': {'predictedLabel': '0', 'predictedScores': {'0': 0.0024830158799886703}, 'details': {'Algorithm': 'SGD', 'PredictiveModelType': 'BINARY'}}}


In [107]:
dict_str = data_dict(s)

13734


In [84]:
s = reddit.submission(id="861dui")
print(s.author)
print(s.author.link_karma)
print(s.author.comment_karma)
print(s.author.created_utc)

TheDStudge
61532
42866
1392063895.0


In [49]:
calls = 0
e_count=0
data_dict=[]
cur_subreddit = 0
for post_sub_value in post_sub_value_counts:
    for submission in reddit.subreddit(post_sub_value_counts.index[cur_subreddit]).hot(limit=post_sub_value):
        print(post_sub_value_counts.index[cur_subreddit])
        try:
            #target_sub_ID=submission.id_from_url(submission.url)
            #target_sub=reddit.submission(target_sub_ID)
            target_sub = submission
            if(target_sub.subreddit!="HailCorporate"):
                data_input={}
                data_input["PostID"]=target_sub.id
                print("On " + data_input["PostID"])
                data_input["Title Seniment"]=title_sentiment(target_sub.title)
                data_input["Title Brands"]=title_brands(target_sub.title)
                data_input["Author Link Karma"]=target_sub.author.link_karma
                data_input["Author Comment Karma"]=target_sub.author.comment_karma
                data_input["Author Age"]=(time.time()-target_sub.author.created_utc)/86400
                commentList=[]
                submission.comments.replace_more(limit=0)
                i = 0
                for top_com in target_sub.comments:
                    commentList.append(top_com)
                    i += 1
                    if i > 50:
                        break
                data_input["Avg Comment Sentiment"]=avg_comment_sentiment(commentList)
                data_input["Avg Comment Brand Count"]=avg_comment_brands(commentList)
                data_dict.append(data_input)
                print('Succeeded: ' + data_input["PostID"])
                print(calls)
        except Exception as e:
            print("Excepted " + str(target_sub.id) + " " + str(e))
            print(calls)
            continue
        #data_input["Comments"]=commentList
    cur_subreddit += 1
df_x_false = pd.DataFrame.from_dict(data_dict)

pics
On 86tvyu
Succeeded: 86tvyu
168
pics
On 86usg2
Succeeded: 86usg2
480
pics
On 86vd21
Succeeded: 86vd21
792
pics
On 86v8f2
Succeeded: 86v8f2
1104
pics
On 86v6k9
Succeeded: 86v6k9
1416
pics
On 86uoiy
Succeeded: 86uoiy
1728
pics
On 86u1tv
Succeeded: 86u1tv
2010
pics
On 86szpj
Succeeded: 86szpj
2322
pics
On 86u1m0
Succeeded: 86u1m0
2580
funny
On 815hyt
Succeeded: 815hyt
2670
funny
On 86w5oo
Succeeded: 86w5oo
2868
funny
On 86usrj
Succeeded: 86usrj
3180
funny
On 86v48y
Succeeded: 86v48y
3390
funny
On 86vks5
Succeeded: 86vks5
3654
todayilearned
On 86w51q
Succeeded: 86w51q
3822
todayilearned
On 86u30i
Succeeded: 86u30i
4134
todayilearned
On 86vl1d
Succeeded: 86vl1d
4344
todayilearned
On 86t8yp
Succeeded: 86t8yp
4614
videos
On 86uu90
Succeeded: 86uu90
4926
videos
On 86twa5
Succeeded: 86twa5
5238
videos
On 86srxh
Succeeded: 86srxh
5550
xboxone
On 85lh0w
Succeeded: 85lh0w
5862
xboxone
On 86tcqc
Succeeded: 86tcqc
6174
mildlyinteresting
On 86u3sr
Succeeded: 86u3sr
6486
mildlyinteresting
On 86up

In [30]:
def avg_comment_sentiment(comment_list):
    '''
    MUST HANDLE TOP MODERATOR COMMENT BEFORE CALLING
    Params:
        List of comment objects
    Returns:
        Weighted average of comments
    '''
    client = boto3.client('comprehend')
    comment_sentiment_list = []
    for comment in comment_list:
        response = client.detect_sentiment(
            Text=comment.body[0:299],
            LanguageCode='en'
        )
        global calls
        calls += 3
        if response['Sentiment']=='NEUTRAL':
            sentiment = 0
        elif response['Sentiment']=='POSITIVE':
            sentiment = 1
        elif response['Sentiment']=='NEGATIVE':
            sentiment = -1
        else:
            sentiment = 0
        comment_sentiment_list.append(comment.ups * sentiment)
    return sum(comment_sentiment_list) / len(comment_sentiment_list)

def avg_comment_brands(comment_list):
    ''' 
    Params:
        List of comment objects
    Returns:
        Weighted average of brand count
    '''
    client = boto3.client('comprehend')
    comment_numbrands_list = []
    for comment in comment_list:
        response = client.detect_entities(
            Text=comment.body[0:299],
            LanguageCode='en'
        )
        global calls
        calls += 3
        organization_count = 0
        for ent in response['Entities']:
            if ent['Type'] == 'ORGANIZATION':
                #print(ent['Text'])
                organization_count += 1
        comment_numbrands_list.append(comment.ups * organization_count)
    return sum(comment_numbrands_list)/ len(comment_numbrands_list)

def title_sentiment(title):
    ''' 
    MUST HANDLE TOP MODERATOR COMMENT BEFORE CALLING
    Params:
        List of comment objects
    Returns:
        Weighted average of comments
    '''
    client = boto3.client('comprehend')
    comment_sentiment_list = []
    response = client.detect_sentiment(
        Text=title[0:299],
        LanguageCode='en'
    )
    global calls
    calls += 3
    if response['Sentiment']=='NEUTRAL':
        sentiment = 0
    elif response['Sentiment']=='POSITIVE':
        sentiment = 1
    elif response['Sentiment']=='NEGATIVE':
        sentiment = -1
    else:
        sentiment = 0
    return sentiment

def title_brands(title):
    ''' 
    Params:
        List of comment objects
    Returns:
        Weighted average of brand count
    '''
    client = boto3.client('comprehend')
    comment_numbrands_list = []
    response = client.detect_entities(
        Text=title[0:299],
        LanguageCode='en'
    )
    global calls
    calls += 3
    organization_count = 0
    for ent in response['Entities']:
        if ent['Type'] == 'ORGANIZATION':
            organization_count += 1
    return organization_count